### 0. Utilities

#### Custom types

In [382]:
from enum import Enum

# Industry clasification standards (ICSs)
class IndustryStandard(Enum):
    NACE = "NACE"
    ISIC = "ISIC"
    WZ = "WZ"
    SSIC = "SSIC"

#### Constants

In [383]:
# ICSs as a list
ind_stds = [std.value for std in IndustryStandard]

# Roles of companies in industrial symbiosis
company_roles = (("Providing", "Provider"), ("Intermediate", "Intermediary"), ("Receiving", "Receiver"))

# Standard code difference threshold
DIFF_THRESHOLD = 10

#### Helper functions

##### Obtaining column names

In [384]:
# Obtain column name based on the imported MAESTRI dataset, given ICS and company role
old_col = lambda std, role: f"{std} code - {role} industry"

# Obtain concise column name, given ICS and company role
new_col = lambda std, role: f"{role} {std} code"

# Obtain column name for similarity score, given ICS and company role
similarity_col = lambda std, role: f"{role} {std} code sim. score"

##### Similarity score calculation

**ASSUMPTION:** If the difference of codes of the same level is within a given threshold, then they are equal.

In [385]:
# Ensure the strings are of comparable length
def comparable_strs(lst):
    min_len = min([len(code) for code in lst])
    return [str[:min_len] for str in lst]

# Compare two codes
def compare(code1, code2):
    diff = abs(int(code1) - int(code2))
    return 1 if diff < DIFF_THRESHOLD else 0

# Compare a code with a list of codes
def compare_one_to_many(code1, codes):
    codes = [code1, *codes]
    comparable_codes = comparable_strs(codes)[1:]
    
    scores = [compare(code1, code) for code in comparable_codes]
    return str(sum(scores) / len(scores))

# Compare a list of codes with another list of codes
def compare_many(codes1, codes2):
    lst = [comparable_strs([code1, code2]) for code1, code2 in zip(codes1, codes2)]
    lst = [tuple(l) for l in lst]
    
    scores = [compare(code1, code2) for code1, code2 in lst]
    return str(sum(scores) / len(scores))

In [386]:
# Evaluate similarity score based on NACE code and another standard code
def similarity_score(nace_code_str, std_code_str):
    # If code does not exist
    if nace_code_str == "" or std_code_str == "":
        return ""
    
    # Split the text by either ';' or ','
    
    ## NACE code
    if ";" in nace_code_str:
        nace_codes = nace_code_str.split(";")
    elif "," in nace_code_str:
        nace_codes = nace_code_str.split(",")
    else:
        nace_codes = [nace_code_str]
    
    ## Standard code
    if ";" in std_code_str:
        std_codes = std_code_str.split(";")
    elif "," in std_code_str:
        std_codes = std_code_str.split(",")
    else:
        std_codes = [std_code_str]
    
    if len(nace_codes) > 1:
        if len(std_codes) == 1:
            return compare_one_to_many(std_codes[0], nace_codes)
        elif len(nace_codes) == len(std_codes):
            return compare_many(nace_codes, std_codes)
            
        return ""
    
    return compare_one_to_many(nace_codes[0], std_codes)

### 1. Reading the MAESTRI dataset

#### Importing the spreadsheeet

In [387]:
import pandas as pd

# Read the MAESTRI dataset as a DataFrame
maestri_df = pd.read_excel("data/Exchanges-database Maestri.xlsx", sheet_name="MAESTRI", dtype=str)

# Replace NaN values with empty strings
maestri_df = maestri_df.fillna("")

# Remove carets and asterisks
maestri_df.replace([r"\^|\*|#"], "", regex=True, inplace=True)

#### Split the main dataset into DataFrames for each role (i.e., provider, intermediary, receiver)

In [388]:
# Aggregate relevant column names for data validation
cols_list = [[old_col(std, next(iter(type))) for std in ind_stds] for type in company_roles]

# Obtain subsets within the original dataset for validation
test_dfs = [maestri_df[cols].copy() for cols in cols_list]

# Rename columns within subsets
for i in range(len(company_roles)):
    old_role, new_role = company_roles[i]
    col_dict = dict()
    
    for std in ind_stds:
        k = old_col(std, old_role)
        v = new_col(std, new_role)
        
        col_dict.update({k: v})
    
    test_dfs[i] = test_dfs[i].rename(columns=col_dict)
    
    # Drop rows with null values for the NACE code
    # Source: https://stackoverflow.com/questions/29314033/drop-rows-containing-empty-cells-from-a-pandas-dataframe
    std = ind_stds[i + 1]
    test_dfs[i] = test_dfs[i][   test_dfs[i][new_col(std, new_role)].astype(bool)   ]

test_dfs[1].head()

,Intermediary NACE code,Intermediary ISIC code,Intermediary WZ code,Intermediary SSIC code
28,0150,0150,01500,01412
30,382,382,382,382
31,0610,0610,06100,19201;09001
50,0610,0610,06100,19201;09001
56,1081,1072,10810,10720


### 2. Obtaining similarity scores for validation

In [389]:
# Loop through all company types
for i in range(len(company_roles)):
    # Role: either 'Provider', 'Intermediary' or 'Receiver'
    role = company_roles[i][1]
    
    # NACE column, example: 'Provider NACE code'
    nace_col = new_col(ind_stds[0], role)
    
    # Iterate through all standards except NACE as it is to be compared with
    for std in ind_stds[1:]:
        df = test_dfs[i]
        
        # Standard column, example for ISIC: 'Provider ISIC code'
        std_col = new_col(std, role)
        
        # Zip the NACE and standard columns to iterate through
        tuples = zip(df[nace_col], df[std_col])
        
        # Append the similarity score column of a certain standard to the DataFrame for a given role
        df[similarity_col(std, role)] = [similarity_score(*a) for a in tuples]
        
        # Convert the similarity score column data type to 'float'
        df[similarity_col(std, role)] = df[similarity_col(std, role)].astype(float)
    
    # List containing new order of columns for readability
    cols = [nace_col] + [f(std, role) for std in ind_stds[1:] for f in (new_col, similarity_col)]
    
    # Reorder columns for readability
    test_dfs[i] = test_dfs[i][cols]

In [390]:
test_dfs[0].head()

,Provider NACE code,Provider ISIC code,Provider ISIC code sim. score,Provider WZ code,Provider WZ code sim. score,Provider SSIC code,Provider SSIC code sim. score
0,1920,1920,1.0,19200,1.000000,19201,1.0
1,1920,1920,1.0,19200,1.000000,19201,1.0
2,2410,2410,1.0,24520,0.000000,24310,0.0
3,2410,2410,1.0,24520,0.000000,24310,0.0
4,2351,2394,0.0,23510;23650;23610,0.333333,23940,0.0


In [391]:
test_dfs[1].head()

,Intermediary NACE code,Intermediary ISIC code,Intermediary ISIC code sim. score,Intermediary WZ code,Intermediary WZ code sim. score,Intermediary SSIC code,Intermediary SSIC code sim. score
28,0150,0150,1.0,01500,1.0,01412,1.0
30,382,382,1.0,382,1.0,382,1.0
31,0610,0610,1.0,06100,1.0,19201;09001,0.0
50,0610,0610,1.0,06100,1.0,19201;09001,0.0
56,1081,1072,1.0,10810,1.0,10720,1.0


In [392]:
test_dfs[2].head()

,Receiver NACE code,Receiver ISIC code,Receiver ISIC code sim. score,Receiver WZ code,Receiver WZ code sim. score,Receiver SSIC code,Receiver SSIC code sim. score
0,3511,3510,1.0,35111,1.000000,35101,1.0
1,3511,3510,1.0,35111,1.000000,35101,1.0
2,2351,2394,0.0,23510;23650;23610,0.333333,23940,0.0
3,2351,2394,0.0,23510;23650;23610,0.333333,23940,0.0
4,2361,2395,0.0,28922;46734;23610,0.333333,28243,0.0


### 3. Exporting results to Excel

In [393]:
with pd.ExcelWriter("data/exports/validation.xlsx") as writer:
    for i in range(len(test_dfs)):
        role = company_roles[i][1]
        df = test_dfs[i]
        
        df.to_excel(writer, sheet_name=role, index=False)
        worksheet = writer.sheets[role]
        workbook = writer.book
        
        # Loop through all columns
        for index, col in enumerate(test_dfs[i]):
            series = test_dfs[i][col]
            max_len = max((
                series.astype(str).map(len).max(),  # Length of largest item
                len(str(series.name))               # Length of column name/header
            ))
            
            # Set column width
            worksheet.set_column(index, index, max_len, format)
            
        format = workbook.add_format()
        format.set_align('left')
        format.set_align('vcenter')